In [1]:
#DESCRIPCION
# Trabajo en pandas para facilitar 
# la suma de primas diferencias de riskmanager y datos de aune

##IMPORTS
from argparse import ArgumentParser
import csv
import os, glob, sys
import pandas as pd

In [2]:
##INPUTS

#1) Pide input file y genera el outfile name
# parser = ArgumentParser()
# parser.add_argument('-f','--file', type=str, required=True)
# file = parser.parse_args()

#2) usa el ultimo archivo modificado como entrada. Acepta multiples formatos
extensions_allowed = ['xls','xlsx','csv']
#Cant de inputs
input_amount = 5
#Comentar si solo es 1
# input_amount = input("Cant de archivos de entrada? (Default:1)") 
if not input_amount:
    input_amount = 1
else:
    input_amount = int(input_amount)
# -----
list_of_files = []
file, filename, file_extension = [], [], []
try:
    #Busca todos los archivos con las extensiones pasadas
    for i in range(len(extensions_allowed)):
        list_of_files += glob.glob(r'*.'+extensions_allowed[i])
    #Comprueba que input_amount no sea mayor al tamaño de la lista
    if input_amount > len(list_of_files):
        print(f"Hay {len(list_of_files)} archivos en  {os.getcwdb()}")    
    #Sorted list of latest inputs
    sorted_files = sorted(list_of_files, key=os.path.getctime)
    for i in range(len(list_of_files)):
        file.append(sorted_files[-(i+1)])
        print(f"Leyendo archivo: {file[i]}\n ")
        #Get the extension of file and the root
        filename.append(os.path.splitext(file[i])[0])
        file_extension.append(os.path.splitext(file[i])[1])
        out_file_name = filename[i]
except ValueError as err:
    print(f"ERROR No hay {file_extension} en el directorio actual ({os.getcwdb()})")
    sys.exit(1)

#Ordenamos los archivos
file_sorted = []
filename_sorted = []
file_extension_sorted = []
for f in file:
    if 'Productos por moneda' in f:
        file_sorted.append(f)
        filename_sorted.append(os.path.splitext(f)[0])
        file_extension_sorted.append(os.path.splitext(f)[1])
for f in file:
    if 'Cartera' in f:
        file_sorted.append(f)
        filename_sorted.append(os.path.splitext(f)[0])
        file_extension_sorted.append(os.path.splitext(f)[1])
for f in file:
    if 'export -' in f:
        file_sorted.append(f)
        filename_sorted.append(os.path.splitext(f)[0])
        file_extension_sorted.append(os.path.splitext(f)[1])
for f in file:
    if 'gvRegistryAccount' in f:
        file_sorted.append(f)
        filename_sorted.append(os.path.splitext(f)[0])
        file_extension_sorted.append(os.path.splitext(f)[1])
#Revisamos orden
for f in file_sorted:
    print (f)

##MAIN
#Leemos el archivo traido de AUNE y sacamos Tabla Diferencias Tabla Primas
with open(file_sorted[1]) as file_cartera, \
        open(f'diferencias{file_extension_sorted[1]}', 'w', newline='') as file_cartera_diferencias,\
        open(f'primas{file_extension_sorted[1]}', 'w', newline='') as file_cartera_primas:
    start1 = 0
    end1 = 0
    flag1 = False
#     print(file_cartera)
    reader = csv.reader(file_cartera, delimiter=';')
    orders = list(reader)
    writer = csv.writer(file_cartera_diferencias, delimiter= ';')
    for row in orders:
        #Busco la tabla de Diferencias
        if "Listado de Productos con Interes Abierto, discriminado por comitente" in row:
            # print (reader.line_num)
            start1 = reader.line_num
            flag1 = True
        if flag1:
            # print(row)
            writer.writerow(row)
            if not row:
                end = reader.line_num
                # print (end)
                break
    #Leemos el archivo y sacamos Primas    
    start2 = 0
    end2 = 0
    flag2 = False    
    reader2 = csv.reader(file_cartera, delimiter=';')
    writer2 = csv.writer(file_cartera_primas, delimiter= ';')
    for row in orders:
        #Busco la tabla de Diferencias
        if "Listado de Portfolio detallado, discriminando segun origen: Portfolio Anterior (PA), operado en la Rueda (R) y operaciones Simuladas (S)" in row:
            # print (reader2.line_num)
            start2 = reader2.line_num
            flag2 = True
        if flag2:
            # print(row)
            writer2.writerow(row)
            if not row:
                end = reader2.line_num
                # print (end)
                break

#-----------Comienzo de procesado de datos con pandas-----------------#


Leyendo archivo: diferencias.csv
 
Leyendo archivo: primas.csv
 
Leyendo archivo: export - 2021-08-12T145735.053.xlsx
 
Leyendo archivo: Cartera20210812.csv
 
Leyendo archivo: Productos por moneda.xlsx
 
Leyendo archivo: gvRegistryAccount.xls
 
Productos por moneda.xlsx
Cartera20210812.csv
export - 2021-08-12T145735.053.xlsx
gvRegistryAccount.xls


In [3]:
# Tabla Diferencias. Suma por Comitente y moneda de liquidación "Dif. DÃ­a" (fila N)

#Leemos la tabla sacada arriba con pandas
file = "diferencias.csv"
df = pd.read_csv(file,skiprows=1,quoting=csv.QUOTE_NONE,delimiter=';',decimal=",")
file2 = "Productos por moneda.xlsx"
df2 = pd.read_excel(file2)
mergedStuff = pd.merge(df, df2, on=['Producto'], how='inner')
#Sumamos por comitenente y moneda
df_diferencias = mergedStuff.groupby(['Comitente','Moneda de liquidación '],as_index=False)[['Dif. Día']].sum()

In [4]:
# Tabla Primas. Suma por Comitente y moneda de liquidación "Primas" (fila L)

file3 = "primas.csv"
df = pd.read_csv(file3,skiprows=1,quoting=csv.QUOTE_NONE,delimiter=';',decimal=",")
file2 = "Productos por moneda.xlsx"
df2 = pd.read_excel(file2)
mergedStuff = pd.merge(df, df2, on=['Producto'], how='inner')
#Sumamos por comitenente y moneda
df_primas = mergedStuff.groupby(['Comitente','Moneda de liquidación '],as_index=False)[['Primas']].sum()

In [5]:
#Leer gvRegistryAccount.xls. Archivo de AP5
#Skipeamos 1 linea porque aune siempre tiene la 1linea vacia

file6 = file_sorted[3]
df_ap5 = pd.read_excel(file6)
df_ap5_slim = df_ap5[['Cuenta', 'Comitente CVSA']]

In [6]:
#Leer export - 2021-07-23T152241.834.xlsx. Archivo de AUNE
#Skipeamos 1 linea porque aune siempre tiene la 1linea vacia

file4 = file_sorted[2]
df_aune = pd.read_excel(file4, skiprows=1)
#Drop la ultima fila, ya que es el total y no sirve
df_aune.drop(df_aune.tail(1).index,inplace=True)
#Extraemos el nro entre corchetes de la co Cuenta para compararla desp
df_aune['Comitente CVSA'] =  df_aune.Cuenta.str.extract('.*\[(.*)\].*')
#Pasamos CVSA a float para poder hacer el merge
df_aune["Comitente CVSA"] = pd.to_numeric(df_aune["Comitente CVSA"])
#Valor de USD Garantia ROFEX. Ver como obtenerlo automaticamente no hardcodeado
USD_Rofex = float(input("Ingrese valor USD Garantia ROFEX"))
#Necesito el opuesto de los valores, multiplico por -1
INVERTIR_SIGNO = -1
df_aune['Aune Total Pesificado'] = df_aune['Total']*USD_Rofex*INVERTIR_SIGNO
# Solo dejamos las dos columnas que necesitamos
df_aune_slim = df_aune[['Comitente CVSA', 'Aune Total Pesificado']]
df_aune_slim

/home/luciano/.local/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Ingrese valor USD Garantia ROFEX 96.71


,Comitente CVSA,Aune Total Pesificado
0,10082,718555.300
1,10084,49322.100
2,10121,-48355.000
3,10122,247577.600
4,10127,116052.000
5,8059,834587.958
6,8946,35009.020
7,9061,212568.580
8,9593,126448.325
9,9706,109669.140


In [7]:
#Merge Diferencias y Primas por Comitente y Moneda.
mergedStuff = pd.merge(df_diferencias, df_primas, on=['Comitente','Moneda de liquidación '], how='inner')
#Cambiamos el nombre
mergedStuff = mergedStuff.rename(columns={"Comitente": "Cuenta"})
mergedStuff

,Cuenta,Moneda de liquidación,Dif. Día,Primas
0,11909,ARS,9200.000000,0.0
1,11909,USDL,-124374.965001,-194260.0
2,17629,ARS,3200.000000,0.0
3,17629,USDL,51478.900000,0.0
4,54251,USDL,-291.390000,0.0
5,80722,ARS,34835.000000,0.0
6,101332,ARS,1600.000000,0.0
7,101567,ARS,272739.999998,0.0
8,102123,ARS,-34000.000000,0.0
9,140336,USDL,48565.000000,331213.3


In [8]:
#Correlacionamos los comitenes de AP5 a su numero en AUNE
mergedStuff3 = pd.merge(mergedStuff,df_ap5_slim, on=['Cuenta'], how='inner')
mergedStuff3

,Cuenta,Moneda de liquidación,Dif. Día,Primas,Comitente CVSA
0,11909,ARS,9200.000000,0.0,3122.0
1,11909,USDL,-124374.965001,-194260.0,3122.0
2,17629,ARS,3200.000000,0.0,3192.0
3,17629,USDL,51478.900000,0.0,3192.0
4,54251,USDL,-291.390000,0.0,8946.0
5,80722,ARS,34835.000000,0.0,9732.0
6,101332,ARS,1600.000000,0.0,9785.0
7,101567,ARS,272739.999998,0.0,9601.0
8,102123,ARS,-34000.000000,0.0,9217.0
9,140336,USDL,48565.000000,331213.3,10082.0


In [9]:
#Separamos la tabla entre ARS y USDL
mergedStuff3_ars = mergedStuff3[mergedStuff3['Moneda de liquidación '] == 'ARS']
mergedStuff3_usdl = mergedStuff3[mergedStuff3['Moneda de liquidación '] == 'USDL']

In [10]:
#Merge tabla de USDL coon aune-slim
mergedStuff4 = pd.merge(mergedStuff3_usdl,df_aune_slim, on=['Comitente CVSA'], how='outer')
#Llenamos los NaN con 0
mergedStuff4 = mergedStuff4.fillna(0)

In [11]:
# Hacemos el total
mergedStuff4['Super Total'] = mergedStuff4['Dif. Día']+mergedStuff4['Primas']+mergedStuff4['Aune Total Pesificado']
mergedStuff4

,Cuenta,Moneda de liquidación,Dif. Día,Primas,Comitente CVSA,Aune Total Pesificado,Super Total
0,11909.0,USDL,-124374.965001,-194260.0,3122.0,0.000,-3.186350e+05
1,17629.0,USDL,51478.900000,0.0,3192.0,0.000,5.147890e+04
2,54251.0,USDL,-291.390000,0.0,8946.0,35009.020,3.471763e+04
3,140336.0,USDL,48565.000000,331213.3,10082.0,718555.300,1.098334e+06
4,140338.0,USDL,-3885.200000,44679.8,10084.0,49322.100,9.011670e+04
5,143317.0,USDL,53421.500000,-53421.5,10127.0,116052.000,1.160520e+05
6,143515.0,USDL,-86445.700000,0.0,10121.0,-48355.000,-1.348007e+05
7,143516.0,USDL,86445.700000,0.0,10122.0,247577.600,3.340233e+05
8,0.0,0,0.000000,0.0,8059.0,834587.958,8.345880e+05
9,0.0,0,0.000000,0.0,9061.0,212568.580,2.125686e+05


In [12]:
#Hacemos 0 los positivos del Super Total
for i in mergedStuff4.iloc[:,mergedStuff4.columns.get_loc("Super Total"):mergedStuff4.columns.get_loc("Super Total")+4]<0:
    for index, j in enumerate(mergedStuff4[i]):
        if j>0:
            mergedStuff4.at[index, i] = 0
mergedStuff4

,Cuenta,Moneda de liquidación,Dif. Día,Primas,Comitente CVSA,Aune Total Pesificado,Super Total
0,11909.0,USDL,-124374.965001,-194260.0,3122.0,0.000,-318634.965001
1,17629.0,USDL,51478.900000,0.0,3192.0,0.000,0.000000
2,54251.0,USDL,-291.390000,0.0,8946.0,35009.020,0.000000
3,140336.0,USDL,48565.000000,331213.3,10082.0,718555.300,0.000000
4,140338.0,USDL,-3885.200000,44679.8,10084.0,49322.100,0.000000
5,143317.0,USDL,53421.500000,-53421.5,10127.0,116052.000,0.000000
6,143515.0,USDL,-86445.700000,0.0,10121.0,-48355.000,-134800.700000
7,143516.0,USDL,86445.700000,0.0,10122.0,247577.600,0.000000
8,0.0,0,0.000000,0.0,8059.0,834587.958,0.000000
9,0.0,0,0.000000,0.0,9061.0,212568.580,0.000000


In [13]:
#Agarramos los ARS de Diferencias y Primas
mergedStuff3_ars = mergedStuff3_ars.rename(columns={"Dif. Día": "Dif. Día ars", "Primas": "Primas ars"})
mergedStuff3_ars
mergedStuff3_ars = mergedStuff3_ars[['Comitente CVSA', 'Dif. Día ars', 'Primas ars']]
mergedStuff3_ars

,Comitente CVSA,Dif. Día ars,Primas ars
0,3122.0,9200.000000,0.0
2,3192.0,3200.000000,0.0
5,9732.0,34835.000000,0.0
6,9785.0,1600.000000,0.0
7,9601.0,272739.999998,0.0
8,9217.0,-34000.000000,0.0
11,9719.0,-8500.000000,0.0


In [14]:
#Agregamos las columnas de Dif. Día ars y Primas ars
mergedStuff5 = pd.merge(mergedStuff3_ars, mergedStuff4, on=['Comitente CVSA'], how='outer')
mergedStuff5 = mergedStuff5.fillna(0)
mergedStuff5

,Comitente CVSA,Dif. Día ars,Primas ars,Cuenta,Moneda de liquidación,Dif. Día,Primas,Aune Total Pesificado,Super Total
0,3122.0,9200.000000,0.0,11909.0,USDL,-124374.965001,-194260.0,0.000,-318634.965001
1,3192.0,3200.000000,0.0,17629.0,USDL,51478.900000,0.0,0.000,0.000000
2,9732.0,34835.000000,0.0,0.0,0,0.000000,0.0,0.000,0.000000
3,9785.0,1600.000000,0.0,0.0,0,0.000000,0.0,0.000,0.000000
4,9601.0,272739.999998,0.0,0.0,0,0.000000,0.0,0.000,0.000000
5,9217.0,-34000.000000,0.0,0.0,0,0.000000,0.0,0.000,0.000000
6,9719.0,-8500.000000,0.0,0.0,0,0.000000,0.0,0.000,0.000000
7,8946.0,0.000000,0.0,54251.0,USDL,-291.390000,0.0,35009.020,0.000000
8,10082.0,0.000000,0.0,140336.0,USDL,48565.000000,331213.3,718555.300,0.000000
9,10084.0,0.000000,0.0,140338.0,USDL,-3885.200000,44679.8,49322.100,0.000000


In [15]:
#Sumamos a Super Total los Dif. Día ars	Primas ars
mergedStuff5['Super Total'] = mergedStuff5['Dif. Día ars'] + mergedStuff5['Primas ars'] + mergedStuff5['Super Total']

mergedStuff5

,Comitente CVSA,Dif. Día ars,Primas ars,Cuenta,Moneda de liquidación,Dif. Día,Primas,Aune Total Pesificado,Super Total
0,3122.0,9200.000000,0.0,11909.0,USDL,-124374.965001,-194260.0,0.000,-309434.965001
1,3192.0,3200.000000,0.0,17629.0,USDL,51478.900000,0.0,0.000,3200.000000
2,9732.0,34835.000000,0.0,0.0,0,0.000000,0.0,0.000,34835.000000
3,9785.0,1600.000000,0.0,0.0,0,0.000000,0.0,0.000,1600.000000
4,9601.0,272739.999998,0.0,0.0,0,0.000000,0.0,0.000,272739.999998
5,9217.0,-34000.000000,0.0,0.0,0,0.000000,0.0,0.000,-34000.000000
6,9719.0,-8500.000000,0.0,0.0,0,0.000000,0.0,0.000,-8500.000000
7,8946.0,0.000000,0.0,54251.0,USDL,-291.390000,0.0,35009.020,0.000000
8,10082.0,0.000000,0.0,140336.0,USDL,48565.000000,331213.3,718555.300,0.000000
9,10084.0,0.000000,0.0,140338.0,USDL,-3885.200000,44679.8,49322.100,0.000000


In [24]:
#Saco columnas innecesarias
df_final = mergedStuff5[['Comitente CVSA','Super Total']]
df_final

,Comitente CVSA,Super Total
0,3122.0,-309434.965001
1,3192.0,3200.000000
2,9732.0,34835.000000
3,9785.0,1600.000000
4,9601.0,272739.999998
5,9217.0,-34000.000000
6,9719.0,-8500.000000
7,8946.0,0.000000
8,10082.0,0.000000
9,10084.0,0.000000


In [31]:
df_final= df_final[df_final !=0]
df_final = df_final.dropna()
df_final

,Comitente CVSA,Super Total
0,3122.0,-309434.965001
1,3192.0,3200.000000
2,9732.0,34835.000000
3,9785.0,1600.000000
4,9601.0,272739.999998
5,9217.0,-34000.000000
6,9719.0,-8500.000000
11,10121.0,-134800.700000


In [20]:
df_final.to_csv('resumen_de_cuenta.csv', sep=';',header=True, index=False, decimal=',')

In [32]:
print("Total:",df_final['Super Total'].sum())

Total: -174360.66500381863
